<a href="https://colab.research.google.com/github/janmejay484/real-time-industry-insight-ai/blob/main/Realtime_Industry_Insight_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pandas matplotlib


In [2]:
!pip install openai tweepy prophet

In [3]:
import openai, tweepy, pandas as pd, matplotlib.pyplot as plt

In [4]:
print("✅ Environment ready!")

✅ Environment ready!
